In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

Perform a kind of an ablation study:

0) Load California house prices dataset.
Take a look at the data.
[California housing dataset page](https://inria.github.io/scikit-learn-mooc/python_scripts/datasets_california_housing.html)

```
from sklearn.datasets import fetch_california_housing
california_housing = fetch_california_housing(as_frame=True)
california_housing = california_housing.frame
california_housing.head()
```

1) Train KNN Regressor. You may use with grid search to select the hyperparameters. Train the classifier **with and without** feature selection step (on all available features). Is there any change in quality on test data? In prediction speed? Compare the two models` quality. 

2) Train Linear regression (Ridge, Lasso) **with and without** feature normalization/scaling step. How does that affect the model`s quality on test data? Compare the two models quality.

3) Train Linear regression **with and without** feature selection step. How does that affect the model`s quality on test data? If you  try to filter even more features, how would that affect the model performance? How will coefficients change?

4) (**optional**) Implement a brute force KNN Algorithm (without using classes from sklearn or any other solutions). Test it on Boston or California, Iris datasets. 

---
### Task 0. Loading *California house prices* dataset

In [2]:
from sklearn.datasets import fetch_california_housing
california_housing = fetch_california_housing(as_frame=True)
california_housing = california_housing.frame
california_housing.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


---
## Task 1. KNN Regressor

In [3]:
X = california_housing[california_housing.columns.to_list()].drop(columns=['MedHouseVal']).values
Y = california_housing['MedHouseVal'].values
print(f'TRAINING DATA: {X}')
print('***')
print(f'LABELS: {Y}')

TRAINING DATA: [[   8.3252       41.            6.98412698 ...    2.55555556
    37.88       -122.23      ]
 [   8.3014       21.            6.23813708 ...    2.10984183
    37.86       -122.22      ]
 [   7.2574       52.            8.28813559 ...    2.80225989
    37.85       -122.24      ]
 ...
 [   1.7          17.            5.20554273 ...    2.3256351
    39.43       -121.22      ]
 [   1.8672       18.            5.32951289 ...    2.12320917
    39.43       -121.32      ]
 [   2.3886       16.            5.25471698 ...    2.61698113
    39.37       -121.24      ]]
***
LABELS: [4.526 3.585 3.521 ... 0.923 0.847 0.894]


In [4]:
# Splitting the dataset into training and validating subsets
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, shuffle=True, random_state=1)

In [5]:
x_train.shape, x_test.shape

((16512, 8), (4128, 8))

In [6]:
y_train.shape, y_test.shape

((16512,), (4128,))

### 1.1 No feature selection

In [7]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV

In [8]:
param_grid = {'n_neighbors': list(range(1, 10, 2)),
              'weights':['uniform', 'distance'],
              'metric':['minkowski','cosine', 'euclidean','cityblock']}

In [141]:
grid_knn = GridSearchCV(KNeighborsRegressor(), param_grid=param_grid, refit=True, verbose=3)
grid_knn.fit(x_train, y_train)

Fitting 5 folds for each of 40 candidates, totalling 200 fits
[CV 1/5] END metric=minkowski, n_neighbors=1, weights=uniform;, score=-0.311 total time=   0.0s
[CV 2/5] END metric=minkowski, n_neighbors=1, weights=uniform;, score=-0.194 total time=   0.0s
[CV 3/5] END metric=minkowski, n_neighbors=1, weights=uniform;, score=-0.251 total time=   0.0s
[CV 4/5] END metric=minkowski, n_neighbors=1, weights=uniform;, score=-0.284 total time=   0.0s
[CV 5/5] END metric=minkowski, n_neighbors=1, weights=uniform;, score=-0.241 total time=   0.0s
[CV 1/5] END metric=minkowski, n_neighbors=1, weights=distance;, score=-0.311 total time=   0.0s
[CV 2/5] END metric=minkowski, n_neighbors=1, weights=distance;, score=-0.194 total time=   0.0s
[CV 3/5] END metric=minkowski, n_neighbors=1, weights=distance;, score=-0.251 total time=   0.0s
[CV 4/5] END metric=minkowski, n_neighbors=1, weights=distance;, score=-0.284 total time=   0.0s
[CV 5/5] END metric=minkowski, n_neighbors=1, weights=distance;, score

GridSearchCV(estimator=KNeighborsRegressor(),
             param_grid={'metric': ['minkowski', 'cosine', 'euclidean',
                                    'cityblock'],
                         'n_neighbors': [1, 3, 5, 7, 9],
                         'weights': ['uniform', 'distance']},
             verbose=3)

In [142]:
grid_knn.best_params_

{'metric': 'cosine', 'n_neighbors': 9, 'weights': 'distance'}

In [143]:
# R-squared score for the model with the best params after fitting without feature selection
grid_knn.score(x_test, y_test)

0.45648635897426293

### 1.2 Feature selection
**Several approaches to a feature selection stage will be represented below:**

a) **correlation-based method** - *based on features correlation with the target variable, ones with a high correlation are selected, and ones with a low correlation are removed*

b) **embedded method** - *the model learns which features are important during training*

#### a) Correlation-based method

In [19]:
# Number of all available features
num_features = range(1, len(california_housing.columns))

In [34]:
from sklearn.feature_selection import SelectKBest, f_regression

corr_models = []

for k_feat in num_features:
    # Selecting the top K features using f_regression as the scoring function
    selector_corr = SelectKBest(f_regression, k=k_feat)  # The number of features to train on will not be strictly predefined - several models with different value of k will be trained 
    x_train_kbest = selector_corr.fit_transform(x_train, y_train)
    
    # Creating and train a KNeighborsRegressor model on the selected features with grid search
    knn_regressor = GridSearchCV(KNeighborsRegressor(), param_grid=param_grid, refit=True, verbose=3)
    knn_regressor.fit(x_train_kbest, y_train)
    
    # Evaluating the model on the test data
    score_corr = knn_regressor.score(selector_corr.transform(x_test), y_test)
    
    corr_set = (knn_regressor, score_corr, selector_corr)
    corr_models.append(corr_set)
    
    print(f'R-squared score on test data: {score_corr}')

Fitting 5 folds for each of 40 candidates, totalling 200 fits
[CV 1/5] END metric=minkowski, n_neighbors=1, weights=uniform;, score=-0.058 total time=   0.0s
[CV 2/5] END metric=minkowski, n_neighbors=1, weights=uniform;, score=0.035 total time=   0.0s
[CV 3/5] END metric=minkowski, n_neighbors=1, weights=uniform;, score=-0.053 total time=   0.0s
[CV 4/5] END metric=minkowski, n_neighbors=1, weights=uniform;, score=-0.034 total time=   0.0s
[CV 5/5] END metric=minkowski, n_neighbors=1, weights=uniform;, score=0.038 total time=   0.0s
[CV 1/5] END metric=minkowski, n_neighbors=1, weights=distance;, score=-0.058 total time=   0.0s
[CV 2/5] END metric=minkowski, n_neighbors=1, weights=distance;, score=0.035 total time=   0.0s
[CV 3/5] END metric=minkowski, n_neighbors=1, weights=distance;, score=-0.053 total time=   0.0s
[CV 4/5] END metric=minkowski, n_neighbors=1, weights=distance;, score=-0.034 total time=   0.0s
[CV 5/5] END metric=minkowski, n_neighbors=1, weights=distance;, score=0.

In [35]:
# Finding the model with best parameters after feature selection
best_corr_model_res = max(corr_models, key=lambda x: x[1])
best_corr_model = best_corr_model_res[0]
best_corr_selector = best_corr_model_res[2]
print(f'BEST PARAMETERS -> {best_corr_model.best_params_}\nBEST NUM_FEATURES -> {best_corr_model.n_features_in_}')

BEST PARAMETERS -> {'metric': 'cityblock', 'n_neighbors': 9, 'weights': 'distance'}
BEST NUM_FEATURES -> 6


In [149]:
# R-squared score for the model with the best params after fitting with correlation-based feature selection
best_corr_model.score(best_corr_selector.transform(x_test), y_test)

0.6749032135129607

#### b) Embedded method

In [31]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso

# Creating and training a Lasso model
lasso = Lasso(alpha=0.01)
lasso.fit(x_train, y_train)

# Creating a SelectFromModel object with the trained Lasso model
selector_emb = SelectFromModel(lasso, prefit=True)

# Transforming the train data using the SelectFromModel object
x_train_embedded = selector_emb.transform(x_train)

# Creating and training a KNeighborsRegressor model on the selected features
knn_regressor_emb = GridSearchCV(KNeighborsRegressor(), param_grid=param_grid, refit=True, verbose=3)
knn_regressor_emb.fit(x_train_embedded, y_train)

# Evaluating the model on the test data
score_emb = knn_regressor_emb.score(selector_emb.transform(x_test), y_test)
print("R-squared score on test data: ", score_emb)

Fitting 5 folds for each of 40 candidates, totalling 200 fits
[CV 1/5] END metric=minkowski, n_neighbors=1, weights=uniform;, score=0.548 total time=   0.0s
[CV 2/5] END metric=minkowski, n_neighbors=1, weights=uniform;, score=0.576 total time=   0.0s
[CV 3/5] END metric=minkowski, n_neighbors=1, weights=uniform;, score=0.528 total time=   0.0s
[CV 4/5] END metric=minkowski, n_neighbors=1, weights=uniform;, score=0.566 total time=   0.1s
[CV 5/5] END metric=minkowski, n_neighbors=1, weights=uniform;, score=0.552 total time=   0.0s
[CV 1/5] END metric=minkowski, n_neighbors=1, weights=distance;, score=0.548 total time=   0.0s
[CV 2/5] END metric=minkowski, n_neighbors=1, weights=distance;, score=0.576 total time=   0.0s
[CV 3/5] END metric=minkowski, n_neighbors=1, weights=distance;, score=0.528 total time=   0.0s
[CV 4/5] END metric=minkowski, n_neighbors=1, weights=distance;, score=0.566 total time=   0.0s
[CV 5/5] END metric=minkowski, n_neighbors=1, weights=distance;, score=0.552 to

In [39]:
print(f'BEST PARAMETERS -> {knn_regressor_emb.best_params_}\nBEST NUM_FEATURES -> {knn_regressor_emb.n_features_in_}')

BEST PARAMETERS -> {'metric': 'cityblock', 'n_neighbors': 9, 'weights': 'distance'}
BEST NUM_FEATURES -> 7


In [151]:
# R-squared score for the model with the best params after fitting with embedded feature selection
knn_regressor_emb.score(selector_emb.transform(x_test), y_test)

0.7332634512357257

### 1.3 To sum up...

- The most accurate results were obtained from the model with embedded feature selection. 
- The conducted experiments proved **feature selection** step to be **crusial** in terms of KNN Regressor performance. 
- Furthermore, as two different approaches to feature selection were implemented, we will consider the latter of them to be more **efficient (embedded method)** with **R^2-score=0.73**.

---
## Task 2. Linear regression
Here four different linear regression models will be trained:
1) **Linear Regression** - basic linear regression model that tries to fit the data as close as possible without any regularization
2) **Ridge Regression (L2 Regularization)** - adds a L2 regularization term to the loss function, which is the sum of squared coefficients
3) **Lasso (L1 Regularization)** - adds a L1 regularization term to the loss function, which is the absolute value of the coefficients
4) **Elastic Net** - a combination of Ridge and Lasso regularization

***!NB** regularization terms are introduced to prevent models from overfitting*

### 2.1 No feature normalization/scaling

In [112]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import r2_score

# Initializing models
lin_reg = LinearRegression()
ridge_reg = Ridge()
lasso_reg = Lasso()
elastic_net = ElasticNet()

# Using grid search, we will define the most suitable value of alpha - a regularization hyperparameter that controls the magnitude of the penalty term applied to the regression coefficients 

alphas = np.array([0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50, 100])

grid_ridge_reg = GridSearchCV(ridge_reg, param_grid=dict(alpha=alphas))
grid_ridge_reg.fit(x_train, y_train)

grid_lasso_reg = GridSearchCV(lasso_reg, param_grid=dict(alpha=alphas))
grid_lasso_reg.fit(x_train, y_train)

grid_elastic_net = GridSearchCV(elastic_net, param_grid=dict(alpha=alphas))
grid_elastic_net.fit(x_train, y_train)

GridSearchCV(estimator=ElasticNet(),
             param_grid={'alpha': array([1.e-02, 5.e-02, 1.e-01, 5.e-01, 1.e+00, 5.e+00, 1.e+01, 5.e+01,
       1.e+02])})

In [105]:
# Getting the best alphas
best_alpha_ridge, best_alpha_lasso, best_alpha_elastic = grid_ridge_reg.best_params_['alpha'], grid_lasso_reg.best_params_['alpha'], grid_elastic_net.best_params_['alpha']

In [114]:
# Training LinReg & training models with the best alpha
lin_reg.fit(x_train, y_train)

ridge_reg = Ridge(alpha=best_alpha_ridge)
ridge_reg.fit(x_train, y_train)

lasso_reg = Lasso(alpha=best_alpha_lasso)
lasso_reg.fit(x_train, y_train)

elastic_net = ElasticNet(alpha=best_alpha_elastic)
elastic_net.fit(x_train, y_train)

ElasticNet(alpha=0.01)

In [121]:
# Predicting the target values
y_pred_linreg = lin_reg.predict(x_test)
y_pred_ridge = ridge_reg.predict(x_test)
y_pred_lasso = lasso_reg.predict(x_test)
y_pred_elastic = elastic_net.predict(x_test)

# Evaluating the models using R2 score
linreg_score = r2_score(y_test, y_pred_linreg)
ridge_score = r2_score(y_test, y_pred_ridge)
lasso_score = r2_score(y_test, y_pred_lasso)
elasticnet_score = r2_score(y_test, y_pred_elastic)

print("Linear Regression R^2 Score:", linreg_score)
print("Ridge R^2 Score:", ridge_score)
print("Lasso R^2 Score:", lasso_score)
print("ElasticNet R^2 Score:", elasticnet_score)

Linear Regression R^2 Score: 0.5965968374812282
Ridge R^2 Score: 0.5963292029985909
Lasso R^2 Score: 0.5936073397889748
ElasticNet R^2 Score: 0.5952354621624334


### 2.2 Feature normalization/scaling

In [109]:
from sklearn.preprocessing import MinMaxScaler

In [111]:
# Normalizing features
scaler = MinMaxScaler()
x_train_transformed = scaler.fit_transform(x_train)
x_test_transformed = scaler.fit_transform(x_test)

In [118]:
# Initializing models
lin_reg_norm = LinearRegression()
ridge_reg_norm = Ridge()
lasso_reg_norm = Lasso()
elastic_net_norm = ElasticNet()

# Using grid search, we will define the most suitable value of alpha - a regularization hyperparameter that controls the magnitude of the penalty term applied to the regression coefficients 

alphas = np.array([0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50, 100])

grid_ridge_reg_norm = GridSearchCV(ridge_reg, param_grid=dict(alpha=alphas))
grid_ridge_reg_norm.fit(x_train_transformed, y_train)

grid_lasso_reg_norm = GridSearchCV(lasso_reg, param_grid=dict(alpha=alphas))
grid_lasso_reg_norm.fit(x_train_transformed, y_train)

grid_elastic_net_norm = GridSearchCV(elastic_net, param_grid=dict(alpha=alphas))
grid_elastic_net_norm.fit(x_train_transformed, y_train)

GridSearchCV(estimator=ElasticNet(alpha=0.01),
             param_grid={'alpha': array([1.e-02, 5.e-02, 1.e-01, 5.e-01, 1.e+00, 5.e+00, 1.e+01, 5.e+01,
       1.e+02])})

In [119]:
# Getting the best alphas
best_alpha_ridge_norm, best_alpha_lasso_norm, best_alpha_elastic_norm = grid_ridge_reg_norm.best_params_['alpha'], grid_lasso_reg_norm.best_params_['alpha'], grid_elastic_net_norm.best_params_['alpha']

In [120]:
# Training LinReg & training models with the best alpha
lin_reg_norm.fit(x_train_transformed, y_train)

ridge_reg_norm = Ridge(alpha=best_alpha_ridge_norm)
ridge_reg_norm.fit(x_train_transformed, y_train)

lasso_reg_norm = Lasso(alpha=best_alpha_lasso_norm)
lasso_reg_norm.fit(x_train_transformed, y_train)

elastic_net_norm = ElasticNet(alpha=best_alpha_elastic_norm)
elastic_net_norm.fit(x_train_transformed, y_train)

ElasticNet(alpha=0.01)

In [123]:
# Predicting the target values
y_pred_linreg_norm = lin_reg_norm.predict(x_test_transformed)
y_pred_ridge_norm = ridge_reg_norm.predict(x_test_transformed)
y_pred_lasso_norm = lasso_reg_norm.predict(x_test_transformed)
y_pred_elastic_norm = elastic_net_norm.predict(x_test_transformed)

# Evaluating the models using R2 score
linreg_score_norm = r2_score(y_test, y_pred_linreg_norm)
ridge_score_norm = r2_score(y_test, y_pred_ridge_norm)
lasso_score_norm = r2_score(y_test, y_pred_lasso_norm)
elasticnet_score_norm = r2_score(y_test, y_pred_elastic_norm)

print("Linear Regression R^2 Score:", linreg_score_norm)
print("Ridge R^2 Score:", ridge_score_norm)
print("Lasso R^2 Score:", lasso_score_norm)
print("ElasticNet R^2 Score:", elasticnet_score_norm)

Linear Regression R^2 Score: 0.5393165058170122
Ridge R^2 Score: 0.5620328778551746
Lasso R^2 Score: 0.5503761720206923
ElasticNet R^2 Score: 0.5240689585019145


### 2.3 To sum up...

- Although in some cases, feature scaling can lead to an increase in the R^2-score by improving the numerical stability and accuracy of the linear regression model, in this particular case traiing **with feature normalization** have shown **worse results**, than without it. 
- It is possible that after feature scaling, the R^2-score of a linear regression model has decreased, because the *coefficients* of the linear regression model, and therefore, the *relationship between the features and the target had been affected*.
- All 4 models' (without feature scaling) scores are almost identical, but **Linear Regression** model's **R^2-score** is the highest and equals to **0.597**

---
## Task 3. Linear regression with feature selection

Basik Linear regression model trained without feature selection turned out to give the highest scores on current dataset, thus, it will be used with feature selection method such as recursive feature elimination (RFE) or univariate feature selection.

In [134]:
# Selecting features based on the magnitude of their coefficients iteratively, choosing the best parameter for feature elimination
from sklearn.feature_selection import RFE

linreg_models = []
for n in num_features:
    linreg_fs = LinearRegression()
    rfe = RFE(linreg_fs, n_features_to_select=n)
    x_train_rfe = rfe.fit_transform(x_train, y_train)
    linreg_fs.fit(x_train_rfe, y_train)
    x_test_rfe = rfe.transform(x_test)
    y_pred_fs = linreg_fs.predict(x_test_rfe)
    lr_score_fs = r2_score(y_test, y_pred_fs)
    linreg_models.append((linreg_fs, lr_score_fs))

In [137]:
for i, model in enumerate(linreg_models, 1):
    print(f'With {i} features LinReg R^2-score is {model[1]}')

With 1 features LinReg R^2-score is -0.0014156997321517206
With 2 features LinReg R^2-score is 0.23411324125903288
With 3 features LinReg R^2-score is 0.5788423577638917
With 4 features LinReg R^2-score is 0.5806375168001883
With 5 features LinReg R^2-score is 0.5877587344849997
With 6 features LinReg R^2-score is 0.5955939876892116
With 7 features LinReg R^2-score is 0.5965749548568093
With 8 features LinReg R^2-score is 0.596596837481235


In [154]:
max(linreg_models, key=lambda x: x[1])

(LinearRegression(), 0.596596837481235)

### To sum up...

- Integrating a **feature selection** step into training out LinReg model **did not help** get any significant improvement in model quality.
- As the **number of features decreases**, the **determination** of the model tends to **decrease** as well.
- In the end, **the best** Linear regression model for California housing dataset is *basik LinReg model without feature normalization trained on all available features*. Its **R^2-score** is **0.597**.

---
## Task 4. KNN Classifier

In [251]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


class KNNClassifier:
    """
    Brute force KNN Algorithm
    """
    def __init__(self, X_train=None, y_train=None, k=None, metric='euclidean'):
        self.X_train = X_train
        self.y_train = y_train
        self.k = k
        self.metric = metric
        
    def _calculate_euclidean_dist(self, X_test):
        distance = np.sqrt(np.sum((self.X_train - X_test[:, np.newaxis])**2, axis=-1))
        return distance

    def _calculate_manhattan_dist(self, X_test):
        distance = np.sum(np.abs(self.X_train - X_test[:, np.newaxis]), axis=-1)
        return distance

    def _calculate_cosine_dist(self, X_test):
        distance = 1 - np.dot(X_test, self.X_train.T) / (np.linalg.norm(X_test, axis=1)[:, np.newaxis] * np.linalg.norm(self.X_train, axis=1))
        return distance
    
    def _calculate_cityblock_distance(self, X_test):
        distance = np.sum(np.abs(X_test[:, np.newaxis] - self.X_train), axis=-1)
        return distance

    def fit(self, data, target):
        self.X_train = data
        self.y_train = target

    def predict(self, X_test):
        if self.metric == 'euclidean':
            distances = self._calculate_euclidean_dist(X_test)
            
        elif self.metric == 'manhattan':
            distances = self._calculate_manhattan_dist(X_test)
            
        elif self.metric == 'cosine':
            distances = self._calculate_cosine_dist(X_test)
            
        elif self.metric == 'cityblock':
            distances = self._calculate_cityblock_distance(X_test)
            
        nearest_indices = np.argsort(distances, axis=1)[:, :self.k]
        nearest_labels = self.y_train[nearest_indices]
        predictions = np.apply_along_axis(lambda x: np.mean(x), axis=1, arr=nearest_labels)
        return predictions
        
    def evaluate_classification(self, x_test, y_true):
        y_pred = self.predict(x_test)
        
        tp = np.sum((y_true == 1) & (y_pred == 1))
        tn = np.sum((y_true == 0) & (y_pred == 0))
        fp = np.sum((y_true == 0) & (y_pred == 1))
        fn = np.sum((y_true == 1) & (y_pred == 0))

        accuracy = (tp+tn)/(tp+fp+fn+tn)
        precision = tp / (tp + fp)              
        recall = tp / (tp + fn)
        f1_score = 2 * (precision * recall) / (precision + recall)
        
        evaluation_report = {'accuracy': accuracy, 'precision': precision, 'recall': recall, 'f1_score': f1_score}
        return evaluation_report